# Red neuronal recurrente

### Importando las librerias

In [14]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

### Importando los datos de entrenamiento

In [15]:
dataset_train = open("grimms.txt", 'r', encoding='utf8').read()
# nparray de palabras
characters = list(set(dataset_train)) # debí ordenar esta cosa ;c
char_to_index = { ch:i for i,ch in enumerate(characters) }
index_to_char = { i:ch for i,ch in enumerate(characters) }

### Constantes

In [16]:
SEQ_LENGTH = 50
VOCAB_SIZE = len(characters)
N_SEQUENCES = int(len(dataset_train)/SEQ_LENGTH)
HIDDEN_DIM = 700 # neuronas por capa
LSTM_LAYERS = 3 # numero de capas
DROPOUT_RATIO = 0.3 # dropout de la primera capa

### Creando la estructura de datos de entrenamiento

In [17]:
X_train = np.zeros((N_SEQUENCES, SEQ_LENGTH, VOCAB_SIZE))
y_train = np.zeros((N_SEQUENCES, SEQ_LENGTH, VOCAB_SIZE))
for i in range(0, N_SEQUENCES):
    X_sequence = dataset_train[i*SEQ_LENGTH: (i+1)*SEQ_LENGTH]          # siguente texto de tamaño SEQ_LENGTH
    X_sequence_index = [char_to_index[value] for value in X_sequence]   # mapeo a indices
    input_sequence = np.zeros((SEQ_LENGTH, VOCAB_SIZE))                 # entrada a matriz de categorias
    for j in range(SEQ_LENGTH):
        input_sequence[j][X_sequence_index[j]] = 1                      # probabilidad 1 al caracter dado
    X_train[i] = input_sequence                                         # añadir a entradas del entrenamiento
    
    y_sequence = dataset_train[i*SEQ_LENGTH+1: (i+1)*SEQ_LENGTH+1]      # texto salida (shifteado uno a la derecha)
    y_sequence_index = [char_to_index[value] for value in y_sequence]   # mapeo a indices
    target_sequence = np.zeros((SEQ_LENGTH, VOCAB_SIZE))                # salida a matriz de categorias
    for j in range(SEQ_LENGTH):                                         
        target_sequence[j][y_sequence_index[j]] = 1                     # probabilidad 1 al caracter dado
    y_train[i] = target_sequence                                        # añadir a salidas del entrenamiento

### Construyendo la red neuronal recurrente

In [18]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, TimeDistributed, Activation

def build_rnn():
    # Inicializando la red
    model = Sequential()
    
    #Añadiendo la primera LSTM
    model.add(LSTM(
        units = HIDDEN_DIM, 
        return_sequences = True,             # Pasa las secuencias a la siguiente capa encolada LSTM
        input_shape = (None, VOCAB_SIZE)     # Tamaño de la entrada
    ))
    model.add(Dropout(DROPOUT_RATIO))
    
    for i in range(LSTM_LAYERS - 1):
        #Añadiendo las demas LSTM
        model.add(LSTM(
            units = HIDDEN_DIM, 
            return_sequences = True
        ))
    model.add(TimeDistributed(Dense(VOCAB_SIZE)))   # Combina las LSTM para ser compatible con una salida softmax
    model.add(Activation("softmax"))
    
    # Compilando la red
    model.compile(
        optimizer = 'rmsprop', 
        loss = 'categorical_crossentropy'
    )
    return model

### Entrenamiento

In [21]:
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

checkpoint = ModelCheckpoint(
    filepath = "ckpt/seq50/weightsv2.{epoch:02d}-{loss:.2f}.hdf5",
    monitor = "loss",
    period = 5
)

# model = build_rnn()
# Cargando desde otro checkpoint donde cancelé el entrenamiento porque quería ver algo (8
model = load_model("ckpt/seq50/weights.14-0.49.hdf5")
history = model.fit(X_train, y_train, epochs = 140, batch_size = 32, callbacks = [checkpoint])

Epoch 1/140
10352/10352 [==============================] - 327s - loss: 0.4452   
Epoch 2/140
10352/10352 [==============================] - 377s - loss: 0.4065   
Epoch 3/140
10352/10352 [==============================] - 378s - loss: 0.3735   
Epoch 4/140
10352/10352 [==============================] - 346s - loss: 0.3468   
Epoch 5/140
10352/10352 [==============================] - 364s - loss: 0.3267   
Epoch 6/140
10352/10352 [==============================] - 378s - loss: 0.3101   
Epoch 7/140
10352/10352 [==============================] - 367s - loss: 0.2966   
Epoch 8/140
10352/10352 [==============================] - 347s - loss: 0.2860   
Epoch 9/140
10352/10352 [==============================] - 380s - loss: 0.2773   
Epoch 10/140
10352/10352 [==============================] - 382s - loss: 0.2703   
Epoch 11/140
10352/10352 [==============================] - 330s - loss: 0.2635   
Epoch 12/140
10352/10352 [==============================] - 378s - loss: 0.2593   
Epoch 13/140


RuntimeError: Can't decrement id ref count (unable to extend file properly)

### Cargando ultimo modelo
Primero entrene con secuencias de 200 caracteres con 100 epochs todo el día, al volver del trabajo resulta que se había muerto el proceso de python, así que tuve que volver a empezar, pero ahora si le puse checkpoints. A la mitad de la noche se reinició la compu porque tenía actualizaciones y solo llegó a guardar el epoch 60, y dado que no tomé en cuenta el orden del mapeo de caracteres, practicamente volví a empezar. Volvió a entrenar otros 60 epochs, pero me di cuenta que los resultados no eran muy buenos, prácticamente copiaba enunciados completos. Hice otro entrenamiento con secuencias de 50 caracteres y llegó a los 65 epochs porque se me acabó el espacio en disco. Los resultados no son muy coherentes pero estoy conforme y corto de tiempo.

In [23]:
# Cargando modelo con secuencias de 50 caracteres, epoch 65
model = load_model("ckpt/seq50/weightsv2.64-0.22.hdf5")

### Creando textos

In [24]:
length = 500

ix = [np.random.randint(VOCAB_SIZE)]                      # caracter random
y_char = [index_to_char[ix[-1]]]                          # salida que no se usa para nada
X = np.zeros((1, length, VOCAB_SIZE))                     # Matriz tridimensional de entrada
for i in range(length): 
    X[0, i, :][ix[-1]] = 1                                # Añade el ultimo caracter a la entrada
    print(index_to_char[ix[-1]], end="")                  # Imprime el ultimo caracter
    ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1)    # Predice las probabilidades para cada caracter y toma la mayor
    y_char.append(index_to_char[ix[-1]])                  # Añade caracter predecido a la salida

2 ‘What do you want here?--speak off!’ said the cat quite as my little mother
has come home, and has brought you her finger; but the wild man was standing at the hills, into the bed with the
eggs that sprinkled for joy. At last he could bear it no longer; so he gave him a needle, and drew the turnip to the court that was left all that sick up the castle was called Snow-white, and
the star-gazer took his glass, looked up, and said, ‘Now, my
godden a fellow who will see mine.’ Then the king and mo